In [1]:
#coding=utf-8
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import os
import re

import csv
import random

'''type_choose decides the following operation are aimed at mda data or audit data'''
type_choose="mda"

if type_choose !="mda" and type_choose !="audit":
    exit(0)

# First Clean

    A lot of sentences are depareted with page header such as "XXXX年年度报告" and "XX/XXXX", which lead to a great deal of loss of high quality sentences,this step are aim to recognize and clean the noise.  


In [2]:
'''
read_path leads to the raw_data,
write_path leads to the data without noise,
parent_start and end_string are used to recognize the aimed file in raw data.

both mda sentence and audit sentence need this step.
so if you want to deal with mda_data,use the first four line codes,
if you want to deal with audit_data,use the next four line codes.

'''

read_path='../../raw_data/mda_data'
write_path='../../mid_data/mda_clean1'
parent_start=-15
end_string='pdf.txt_total.md'
if type_choose=="audit":
    read_path='../../raw_data/audit_data'
    write_path='../../mid_data/audit_clean1'
    end_string='pdf.txt_part_3.md'

'''relualar expression'''
pattern=re.compile(r"(\d+(\n+)(.*)\d+(\s)?年年度报告全文\n)|(\d+(\s)?年年度报告全文\n)|(\d+(\n+)(.*)\d+(\s)?年年度报告\n)|(\d+(\s)?年年度报告\n)|((\s)*\d+\s\/\s218\s2014\s年年度报告)|(\n\d+\s/)")

'''build blank file if write_path doesn't exist'''
if not os.path.exists(write_path):
    os.makedirs(write_path)
    
    
for parent,dirnames,filenames in os.walk(read_path):       
    filenumber=1
    for filename in filenames:
        
        if filename.endswith(end_string):
           
            main_path=write_path+'/'+ parent[parent_start:] 
            
            '''rebuild the file tree of reading_path'''
            if not os.path.exists(main_path):
                os.makedirs(main_path)
                
            fwrite = open(os.path.join(main_path,filename),'w',encoding='utf-8')
            content=open(os.path.join(parent,filename),'r',encoding='utf-8').read()
            
            '''delete sentence noise with string replacement'''
            while(re.search(pattern,content)!=None):
                content=re.sub( pattern,"", content)
            fwrite.write(content)
            fwrite.close()    
            
            filenumber=filenumber+1

# Second Clean: Sentences Slicing

#### 切句子规则：
- 1.按行读取句子。如果句子格式是大标题，例如：“第X节”，“（一）、”，“一、” 等格式则弃而不用。考虑到诸如“1、”，“a、”这样的小标题后跟的可能是具体条例，因此不删除。
- 2.如果该行数据里不出现“。”，则删除。因为可能是乱码、表格数据等。
- 3.某行数据里若存在“。”，则按照句号进行分句处理。
- 4.所有句子要求长度超过8，如果没有超过则删除。则删掉诸如“由下表所示：”等无意义短句。
#### 其余处理：
- 1.抽取股票id，年份信息，给每份年报拟年报id，给每条句子赋值句子id，统计每份年报的句子总数量。
- 2.准备两种csv文件，第一个csv文件 按照：股票id，年份，年报id，句子id，积极消极待标注区，句子内容写入。另一个统计csv文件写入 股票id，年份，年报id，年报句子总数（不包括审计文件），文件路径。
- 3.处理统计csv时，出现原始数据里由于公司更名或者重复提交年报导致的同一股票代码同一年份多篇年报的现象，这些年报需要特殊标记然后另外处理。

In [2]:
'''
read_path leads to the data after first cleaned ,
write_path leads to the csv file

both mda sentence and audit sentence need this step.
so if you want to deal with mda_data,use the first two line codes,
if you want to deal with audit_data,use the next two line codes.

'''
read_path='../../mid_data/mda_clean1'
write_path='../../mid_data/mda_clean2/'
if type_choose=="audit":
    read_path='../../mid_data/audit_clean1'
    write_path='../../mid_data/audit_clean2/'

pattern=re.compile(r"第\S节|[(]?[一二三四五六七八九十]+[)]?\、|\([一二三四五六七八九十]+\)")


if not os.path.exists(write_path):
    os.makedirs(write_path)
    
'''totalsentence is used to record the sum of all sentences'''
totalsentence=1
fileid=0

fwrite = None
for parent,dirnames,filenames in os.walk(read_path):
    
    listtemp=[]
    for filename in filenames:
        fileid=fileid+1
        
        '''embeding sentence_number for sentences in the same report '''
        sentence_number=0
        content=open(os.path.join(parent,filename),'r',encoding='utf-8')
        
        '''find the year information of annual report from title(consistent number)''' 
        year=re.findall(r"\d+",filename)
        
        '''find the stock number of annual report from parent folder'''
        stockid=parent[-6:]
        
        for line in content: 
            if re.match(pattern,line) or len(line)<8:
                continue
            elif re.search("。",line):
                sentences=line.split("。")
                for sentence in sentences:
                    if len(sentence)<8 or sentence[-1]=='\n':
                        continue
                    else:
                        
                        '''every 8000 statements in a csv file'''
                        if totalsentence%8000==1:
                            if fwrite != None:
                                fwrite.close()
                            fwrite = open(write_path+type_choose+"_sentence"+str(int(totalsentence/8000))+".csv",'a+',encoding='utf-8-sig',newline='')
                        sentence=sentence+'。'
                        sentence_number=sentence_number+1
                        totalsentence=totalsentence+1
                                                    
                        '''stock_id, year， article id，sentence id, unlabelld tag, sentence content'''
                        struct=[stockid,year[0],fileid,sentence_number,"",sentence]
                        csv_write = csv.writer(fwrite,dialect='excel')
                        csv_write.writerow(struct)
                        #print(str(sentence_number)+" "+sentence)
            
                
        '''some row reports are repeated, pick those by stockid and reporting year and record'''     
        flag=0
        struct_temp=[stockid,year[0]]
        '''this report has been exist'''
        if struct_temp in listtemp:
            flag=1
            
        '''each time one report was completed,the statistics are record for further using'''    
        listtemp.append(struct_temp)
        
        if type_choose=='mda':
            with open("../../mid_data/number.csv",'a+',encoding='utf-8-sig',newline='')as fstatistic:
                '''writing item consists stockid  year  file_id sentence_sum，report path'''
                statistic_struct=[stockid,year[0],fileid,sentence_number,parent+"\\"+filename]
                if flag==1:
                    statistic_struct=["Attention",stockid,year[0],fileid,sentence_number,parent+"\\"+filename]

                csv_write = csv.writer(fstatistic,dialect='excel')
                csv_write.writerow(statistic_struct)
             
            
fwrite.close()


# Third:  Cleaned  Repeated Sentences And Shuffle

     第二步把原始数据文件切成了一个个完整的待标记的句子，为了把处理好的数据交付人工标记，还需要删除重复数据并且乱序处理,这一步耗时较多。

In [ ]:
read_path="../../mid_data/mda_clean2"   
write_path="../../mid_data/untagged_mda"
if type_choose=="audit":
    read_path="../../mid_data/audit_clean2"
    write_path="../../mid_data/untagged_audit"
    
'''every csv waitting to tagged contains 180 sentences'''
csv_cap=180
if not os.path.exists(write_path):
    os.mkdir(write_path)
sentences=[]     

'''read in all items'''
for parent,dirnames,filenames in os.walk(read_path):     
    for file in filenames:
        csv_file = csv.reader(open(os.path.join(read_path,file),'r',encoding='utf-8-sig')) 
           
        sentencenum=0;
        for sentence in csv_file:
            sentences.append(sentence) 
            
'''list1 contains only sentence content.''' 
'''list2 records the whole item, including stock number,year, and so on.'''
'''for every item in list "sentences",if the content part has not been appeared in list1,''' 
'''then let the item join in list2,let the whole item join in list1'''
list1=[]
list2=[]
s_num=0
for s in sentences:
    s_num +=1
    print("\r %d" %(s_num),end="")
    if not s[5] in list1:
        list1.append(s[5])
        list2.append(s)
print("")
print("清理前句子总数："+str(len(sentences)))
print("清理后句子总数:"+str(len(list2)))

'''shuffle code'''
random.shuffle(list2)

fwrite = open(os.path.join(write_path,type_choose+"sentence0.csv"),'a+',encoding='utf-8-sig',newline='')
item_number=0
for item in list2:
    if item_number%csv_cap==0:                            
        fwrite.close()
        fwrite = open(os.path.join(write_path,type_choose+"sentence"+str(filename[int(item_number/csv_cap)])+".csv"),'a+',encoding='utf-8-sig',newline='')
    csv_write = csv.writer(fwrite,dialect='excel')
    csv_write.writerow(item)
    item_number=item_number+1